In [2]:
class PipeMap:
    
    s_directionPipeEntryMap = None
    s_directionCoordinateChangeMap = None
    
    def __init__(self, text):
        self._map = text.split('\n')
        self._yBound = len(self._map)
        self._xBound = len(self._map[0])
        self._startingCoordinate = self.initStartingCoordinate()
        self._startingDirections = self.initStartingDirections()
        
    def getMap(self):
        return self._map
        
    def getStartingCoordinate(self):
        return self._startingCoordinate
    
    def getStartingDirections(self):
        return self._startingDirections
        
    def initStartingCoordinate(self):
        for i in range(self._yBound):
            for j in range(self._xBound):
                char = self.getPipeTypeAtCoord((i, j))
                if char == 'S':
                    return (i, j)
        raise Exception("No starting coordinate found")
                
    def initStartingDirections(self):
        directions = []
        directionPipeEntryMap = PipeMap.getDirectionPipeEntryMap()
        directionCoordinateChangeMap = PipeMap.getDirectionCoordinateChangeMap()
        for direction in directionPipeEntryMap:
            coordChange = directionCoordinateChangeMap[direction]
            coord = PipeMap.addCoords(self._startingCoordinate, coordChange)
            if coord[0] < 0 or coord[0] >= self._yBound or coord[1] < 0 or coord[1] >= self._xBound:
                continue
            pipeType = self.getPipeTypeAtCoord(coord)
            pipeTypeList = directionPipeEntryMap[direction]
            if pipeType in pipeTypeList:
                directions.append(direction)
        if len(directions) > 2:
            raise Exception("Too many directions found")
        elif len(directions) < 2:
            raise Exception("Too few directions found")
        return directions
    
    def getDirectionPipeEntryMap():
        if PipeMap.s_directionPipeEntryMap == None:
            PipeMap.s_directionPipeEntryMap = PipeMap.initDirectionPipeEntryMap()
        return PipeMap.s_directionPipeEntryMap
            
    def initDirectionPipeEntryMap():
        map_ = {}
        map_['N'] = ['F', '7', '|']
        map_['E'] = ['J', '7', '-']
        map_['S'] = ['L', 'J', '|']
        map_['W'] = ['L', 'F', '-']
        return map_
    
    def getDirectionCoordinateChangeMap():
        if PipeMap.s_directionCoordinateChangeMap == None:
            PipeMap.s_directionCoordinateChangeMap = PipeMap.initDirectionCoordinateChangeMap()
        return PipeMap.s_directionCoordinateChangeMap
    
    def initDirectionCoordinateChangeMap():
        map_ = {}
        map_['N'] = (-1, 0)
        map_['E'] = (0, 1)
        map_['S'] = (1, 0)
        map_['W'] = (0, -1)
        return map_
    
    def addCoords(coord1, coord2):
        return (coord1[0] + coord2[0], coord1[1] + coord2[1])
    
    def getPipeTypeAtCoord(self, coord):
        return self._map[coord[0]][coord[1]]
        

In [3]:
class PathFollower:
    
    s_mapDirectionForPipeType = None
    
    def __init__(self, map_: PipeMap, direction):
        self._map = map_
        self._coord = map_.getStartingCoordinate()
        self._startingDirection = direction
        self._direction = direction
        self._steps = 0
        
    def getCoord(self):
        return self._coord
    
    def getSteps(self):
        return self._steps
    
    def getPipeTypeAtCoord(self):
        return self._map.getPipeTypeAtCoord(self._coord)
        
    def step(self):
        direction = self._direction
        directionCoordinateChangeMap = PipeMap.getDirectionCoordinateChangeMap()
        coordChange = directionCoordinateChangeMap[direction]
        self.addToCoord(coordChange)
        pipeType = self.getPipeTypeAtCoord()
        self.updateDirectionForPipeType(pipeType)
        self._steps += 1
        
    def addToCoord(self, coord):
        self._coord = PipeMap.addCoords(self._coord, coord)
        
    def updateDirectionForPipeType(self, pipeType):
        if pipeType == 'S':
            self._direction = self._startingDirection
            return
        mapDirectionForPipeType = PathFollower.getMapDirectionForPipeType()
        try:
            self._direction = mapDirectionForPipeType[(self._direction, pipeType)]
        except:
            string = "Tried to enter pipe {0} from direction {1} at coordinate ({2}, {3})".\
            format(pipeType, \
                   self._direction, \
                   self._coord[0], \
                   self._coord[1])
            raise Exception(string)
            
    def getMapDirectionForPipeType():
        if PathFollower.s_mapDirectionForPipeType == None:
            PathFollower.s_mapDirectionForPipeType = PathFollower.initMapDirectionForPipeType()
        return PathFollower.s_mapDirectionForPipeType
    
    def initMapDirectionForPipeType():
        map_ = {}
        map_[('N', '|')] = 'N'
        map_[('S', '|')] = 'S'
        map_[('E', '-')] = 'E'
        map_[('W', '-')] = 'W'
        map_[('N', 'F')] = 'E'
        map_[('W', 'F')] = 'S'
        map_[('E', '7')] = 'S'
        map_[('N', '7')] = 'W'
        map_[('E', 'J')] = 'N'
        map_[('S', 'J')] = 'W'
        map_[('W', 'L')] = 'N'
        map_[('S', 'L')] = 'E'
        return map_
        

In [4]:
pipeMap = PipeMap(text)
pathFollowers = [PathFollower(pipeMap, direction) for direction in pipeMap.getStartingDirections()]
while True:
    for follower in pathFollowers:
        follower.step()
    if pathFollowers[0].getCoord() == pathFollowers[1].getCoord():
        break
print(pathFollowers[0].getSteps())


In [5]:
class Tile:
    
    def __init__(self, pipeType):
        self._pipeType = pipeType
        self._onPath = False
        self._pathString = ""
        
    def getPipeType(self):
        return self._pipeType
        
    def setOnPath(self, tf):
        self._onPath = tf
        self._pathString = ""
        
    def isPathStringEmpty(self):
        return self._onPath or len(self._pathString) == 0
        
    def add(self, char):
        if self._onPath:
            return
        if len(self._pathString) > 0:
            lastChar = self._pathString[-1]
            if Tile.areInverses(lastChar, char):
                self._pathString = self._pathString[:-1]
                return
        self._pathString += char
        
    def areInverses(char1, char2):
        return Tile.sameCharDifferentCase(char1, char2)
    
    def sameCharDifferentCase(char1, char2):
        return ord(char1) + Tile.getCaseDiff() == ord(char2) or ord(char2) + Tile.getCaseDiff() == ord(char1)
    
    def getCaseDiff():
        return ord('a') - ord('A')

In [6]:
class TilePipeMap(PipeMap):
    
    def __init__(self, text):
        self._map = [ [ Tile(char) for char in line ] for line in text.split('\n') ]
        self._yBound = len(self._map)
        self._xBound = len(self._map[0])
        self._startingCoordinate = self.initStartingCoordinate()
        self._startingDirections = self.initStartingDirections()
        
    #override
    def getPipeTypeAtCoord(self, coord):
        tile = self._map[coord[0]][coord[1]]
        return tile.getPipeType()
    
    def updateTilesOnAxis(self, coord, direction):
        if direction == 'N' or direction == 'S': 
            self.updateEastWestAxisTiles(coord, direction)
        elif direction == 'E' or direction == 'W':
            self.updateNorthSouthAxisTiles(coord, direction)

    def updateEastWestAxisTiles(self, coord, direction):
        y, x = coord
        for j in range(self._xBound):
            tile = self._map[y][j]
            if j < x:
                char = 'E'
                if direction == 'S':
                    char = char.lower()
                tile.add(char)
            elif j > x:
                char = 'W'
                if direction == 'N':
                    char = char.lower()
                tile.add(char)
            elif j == x:
                tile.setOnPath(True)

    def updateNorthSouthAxisTiles(self, coord, direction):
        y, x = coord
        for i in range(self._yBound):
            tile = self._map[i][x]
            if i < y:
                char = 'S'
                if direction == 'W':
                    char = char.lower()
                tile.add(char)
            elif i > y:
                char = 'N'
                if direction == 'E':
                    char = char.lower()
                tile.add(char)
            elif i == y:
                tile.setOnPath(True)

In [7]:
class TilePathFollower(PathFollower):
    
    #override
    def step(self):
        ## update tiles on axis stepping OFF
        direction = self._direction
        self.updateTilesOnAxis(direction)
        super().step()
        ## update tiles on axis stepping ON
        self.updateTilesOnAxis(direction)
        

    def updateTilesOnAxis(self, direction):
        self._map.updateTilesOnAxis(self._coord, direction)

In [8]:
tilePipeMap = TilePipeMap(text)
startingCoordinate = tilePipeMap.getStartingCoordinate()
direction = tilePipeMap.getStartingDirections()[0] # arbitrary
tilePathFollower = TilePathFollower(tilePipeMap, direction)
while True:
    tilePathFollower.step()
    if tilePathFollower.getCoord() == startingCoordinate:
        break
count = 0
for line in tilePipeMap.getMap():
    for tile in line:
        if not tile.isPathStringEmpty():
            count += 1
print(count)

297
